Exercise 3, Problem 1

In [9]:
import matplotlib.pyplot as plt
import quantecon as qe
import numpy as np
import time
from numba import jit

@jit
def lininterp(x, xp, yp):
    fx = np.zeros_like(x, dtype = float)
    for i in range(len(x)):
        if x[i] <= xp[0]:
            fx[i] = yp[0]
        elif x[i] >= xp[-1]:
            fx[i] = yp[-1]
        else:
            for j in range(len(x)):
                if x[i] > xp[j] and x[i] <= xp[j+1]:
                    slope = (yp[j+1] - yp[j])/(xp[j+1]-xp[j])
                    fx[i] = yp[j] + slope * (x[i] - xp[j])
    return fx

# Check if the function works
x = [3, 5, 2, 10, 13]
xp = [1, 4, 7, 8, 10, 15]
yp = [13, 49, 210, 2, 10, 2]
print(lininterp(x, xp, yp))
print(np.interp(x, xp, yp))

start1 = time.time()
np.interp(x,xp,yp)
end1= time.time()
start2 = time.time()
lininterp(x,xp,yp)
end2= time.time()
print("numpy interpolation:" + str(end1-start1))
print("numba interpolation:" + str(end2-start2))

[  37.          102.66666667   25.           10.            5.2       ]
[  37.          102.66666667   25.           10.            5.2       ]
numpy interpolation:8.392333984375e-05
numba interpolation:9.179115295410156e-05


Exercise 3, Problem 2

In [13]:
## Define the model

alpha = 0.65
beta = 0.95
mu = 0
s = 0.1
grid_min = 1e-6
grid_max = 4
grid_size = 200
shock_size = 250

gamma = 1.5   # Preference parameter
gamma_inv = 1 / gamma

def f(k):
    return k**alpha

def f_prime(k):
    return alpha * k**(alpha - 1)

def u(c):
    return (c**(1 - gamma) - 1) / (1 - gamma)

def u_prime(c):
    return c**(-gamma)

def u_prime_inv(c):
    return c**(-gamma_inv)

k_grid = np.linspace(grid_min, grid_max, grid_size)
shocks = np.exp(mu + s * np.random.randn(shock_size))


def coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
 
    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    
    # Determine endogenous grid
    y = k_grid + c  # y_i = k_i + c_i

    # Update policy function and return
    Kg = lambda x: np.interp(x, y, c)
    return Kg

def crra_coleman_egm(g):
    return coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

sim_length = 20

print("Timing policy function iteration with endogenous grid (numpy interpolation)")
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_egm(g)
    g = new_g
qe.util.toc()

Timing policy function iteration with endogenous grid (numpy interpolation)
TOC: Elapsed: 0.13576173782348633 seconds.


0.13576173782348633

In [17]:
## Define the model

alpha = 0.65
beta = 0.95
mu = 0
s = 0.1
grid_min = 1e-6
grid_max = 4
grid_size = 200
shock_size = 250

gamma = 1.5   # Preference parameter
gamma_inv = 1 / gamma

def f(k):
    return k**alpha

def f_prime(k):
    return alpha * k**(alpha - 1)

def u(c):
    return (c**(1 - gamma) - 1) / (1 - gamma)

def u_prime(c):
    return c**(-gamma)

def u_prime_inv(c):
    return c**(-gamma_inv)

k_grid = np.linspace(grid_min, grid_max, grid_size)
shocks = np.exp(mu + s * np.random.randn(shock_size))


def coleman_numba(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
 
    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    
    # Determine endogenous grid
    y = k_grid + c  # y_i = k_i + c_i

    # Update policy function and return
    Kg = lambda x: lininterp(x, y, c)
    return Kg

def crra_coleman_numba(g):
    return coleman_numba(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

sim_length = 20

print("Timing policy function iteration with endogenous grid (numbafied interpolation)")
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_numba(g)
    g = new_g
qe.util.toc()

Timing policy function iteration with endogenous grid (numbafied interpolation)
TOC: Elapsed: 0.43251776695251465 seconds.


/Users/Sophia/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in power
/Users/Sophia/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in power


0.43251776695251465

So the numbafied linear interpolation function fails to speed up the endogenous grid point method, possibly because I still include a for loop in the function. 